# PetFinder Pawpularity Prediction
## Table of Contents
- Set up
- Import datasets
- Data Preprocessing
- Model Development
    - Tabular Regression Model
- Model Evaluation
- Submission
- Todos

## Set up

In [ ]:
import numpy as np
import pandas as pd

## Import datasets

In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")

## Data Preprocessing

In [ ]:
train.head()

I will add file_path feature for future use.

In [ ]:
train["file_path"] = train["Id"].apply(lambda identifier: "../input/petfinder-pawpularity-score/train/" + identifier + ".jpg")
test["file_path"] = test["Id"].apply(lambda identifier: "../input/petfinder-pawpularity-score/test/" + identifier + ".jpg")

In [ ]:
train.head()

## Model Development

## Submit function

In [ ]:
def submit(model, name):
    sample_submission["Pawpularity"] =  model.predict(test[tabular_columns + statistic_columns])
    sample_submission.to_csv(name, index=False)

## Tabular Regression Model

In [ ]:
tabular_columns = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']

In [ ]:
train.corr()["Pawpularity"]

In [ ]:
for data in [train, test]:
    data["row_mean"] = data[tabular_columns].mean(axis=1)
    data["row_std"] = data[tabular_columns].std(axis=1)
    data["row_num_ones"] = data[tabular_columns].sum(axis=1)
    data["row_num_zeros"] = data["row_num_ones"].apply(lambda item: len(tabular_columns) - item)

In [ ]:
train.head()

In [ ]:
statistic_columns = ["row_mean", "row_std", "row_num_ones", "row_num_zeros"]

In [ ]:
import sklearn
import catboost
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10)
models = []
index = 0
for trian_indices, val_indices in kfold.split(train):
    X_train = train.loc[trian_indices, tabular_columns + statistic_columns]
    y_train = train.loc[trian_indices, "Pawpularity"]
    X_val = train.loc[val_indices, tabular_columns + statistic_columns]
    y_val = train.loc[val_indices, "Pawpularity"]
    print(len(trian_indices), len(val_indices))
    cat = catboost.CatBoostRegressor(
        iterations = 10000, 
        learning_rate = 0.03,
        depth = 5,
        verbose=500,
        early_stopping_rounds=1000,
        eval_metric="RMSE"
    )
    cat.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)
    submit(cat, "submission_fold_%d.csv"%(index+1))
    models.append(cat)

## Submission

In [ ]:
sample_submission["Pawpularity"] =  np.mean([model.predict(test[tabular_columns + statistic_columns]) for model in models], axis=0)
sample_submission.to_csv("submission.csv", index=False)

## Todos
- Develop Image Regression Model
- Combine results of Image Regression Model and Tabular Prediction Model